# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
weather_df = pd.read_csv("new_city_data.csv").reset_index(drop=True).drop(columns="Unnamed: 0")
weather_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind
0,Severodvinsk,64.5635,39.8302,-4.00,84,90,2.24
1,Koran,43.8103,18.5689,44.29,100,75,4.61
2,Butaritari,3.0707,172.7902,81.34,77,100,15.75
3,Goundam,16.4145,-3.6708,81.45,14,8,9.51
4,Ajdabiya,30.7554,20.2263,65.05,27,0,10.51
...,...,...,...,...,...,...,...
541,Lakhdenpokh'ya,61.5198,30.1992,9.59,96,100,3.62
542,Tongren,27.7172,109.1853,48.97,87,86,0.96
543,Padang,-0.9492,100.3543,73.40,88,40,2.30
544,Pokhara,28.2333,83.9833,54.97,48,6,4.47


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)

In [9]:
#Use the Lat and Lng as locations and Humidity as the weight

locations = weather_df[['Latitude', 'Longitude']]
humidity = weather_df['Humidity']

#Add Heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

#new_temp = (weather_df.Temperature < 80) & (weather_df.Temperature > 70)
new_wind = weather_df.Wind < 10
new_cloud = weather_df.Cloudiness == 0
new_criteria = new_wind & new_cloud


new_weather_df = weather_df[new_criteria]
new_weather_df = new_weather_df.dropna()
new_weather_df = new_weather_df.reset_index()
new_weather_df

,index,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind
0,13,Portree,57.4129,-6.1942,27.00,69,0,8.01
1,18,Kafanchan,9.5833,8.3000,74.17,20,0,3.00
2,20,Abéché,13.8292,20.8324,71.01,20,0,5.84
3,22,Kudymkar,59.0173,54.6560,-15.20,88,0,2.66
4,25,La Rioja,42.2500,-2.5000,46.40,71,0,1.14
...,...,...,...,...,...,...,...,...
72,507,Buraidah,26.3260,43.9750,59.00,77,0,6.55
73,516,Yendi,9.4427,-0.0099,85.10,11,0,4.45
74,519,Graaff-Reinet,-32.2522,24.5308,80.01,64,0,5.99
75,522,Kangaatsiaq,68.3065,-53.4641,8.35,86,0,5.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = pd.DataFrame(new_weather_df)

In [ ]:
params = {
    "radius": 5000,
    "types": "lodging"
}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
